In [ ]:
!pip install selenium webdriver_manager
!pip install selenium


In [1]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.alert import Alert
import time 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd 

# Initialize the webdriver
driver = webdriver.Chrome()
driver.get("https://arcweb.hcad.org/parcelviewer/")
ok_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//td[contains(text(),'OK')]"))
)
ok_button.click()

# Click the HTML for the dropdown thingy
dropdown_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//img[@align='left']"))
)
dropdown_button.click()

# Click on the HTML for the address
address_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//td[@value='address']"))
)
address_button.click()

# Create a list of addresses
addresses = [f"{i} Daniel Falls Ln" for i in range(3300, 3420)]

# Create an empty DataFrame to store the results
columns = ['Address', 'HCAD Account', 'Owner Name', 'State Class', 'Appraised Value', 'Market Value']
df = pd.DataFrame(columns=columns)

try:
    for address_to_search in addresses:
        # Animation 2: click the search button
        search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//input[@id='txtAddress']"))
        )
        search_button.click()

        try:
            # Find the input element within the search bar
            input_element_xpath = "//input[@id='txtAddress']"

            # Use an explicit wait to wait for the input element to be present and clickable
            input_element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, input_element_xpath))
            )

            # Input the address into the search bar
            input_element.clear()
            time.sleep(2)  # Adjust this wait time as needed
            input_element.send_keys(address_to_search)
            input_element.send_keys(Keys.RETURN)  # Simulate hitting the "Enter" key

            # Wait for the search result to load (adjust the timeout as needed)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'divParcelDataScrollContainer')))

            # Check if the table is present
            table_present = EC.presence_of_element_located((By.XPATH, "//*[@id='tableParcelInfoWindowData']"))
            if not table_present(driver):
                print(f"No table found for address: {address_to_search}")
                continue  # Move on to the next address if no table is found

            # Extract information from the specified element
            div_table_xpath = "//*[@id='tableParcelInfoWindowData']"

            for _ in range(3):  # Adjust the number of retries as needed
                try:
                    div_table = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, div_table_xpath))
                    )

                    # Extract specific information from the table
                    hcad_account = div_table.find_element(By.XPATH, ".//td[contains(text(), 'HCAD Account:')]/following-sibling::td").text
                    owner_name = div_table.find_element(By.XPATH, ".//td[contains(text(), 'Owner Name:')]/following-sibling::td").text       
                    state_class = div_table.find_element(By.XPATH, ".//td[contains(text(), 'State Class:')]/following-sibling::td").text
                    appraised_value = div_table.find_element(By.XPATH, ".//td[contains(text(), 'Appraised Value:')]/following-sibling::td").text
                    market_value = div_table.find_element(By.XPATH, ".//td[contains(text(), 'Market Value:')]/following-sibling::td").text

                    # Print the extracted information
                    print("\nAddress:", address_to_search)
                    print("HCAD Account:", hcad_account)
                    print("Owner Name:", owner_name)
                    print("State Class:", state_class)
                    print("Appraised Value:", appraised_value)
                    print("Market Value:", market_value)

                    # Append the data to the DataFrame
                    df = df.append({
                        'Address': address_to_search,
                        'HCAD Account': hcad_account,
                        'Owner Name': owner_name,
                        'State Class': state_class,
                        'Appraised Value': appraised_value,
                        'Market Value': market_value
                    }, ignore_index=True)

                    break  # Exit the retry loop if successful

                except StaleElementReferenceException:
                    print("StaleElementReferenceException occurred. Retrying...")
                    time.sleep(2)  # Adjust this wait time as needed

        except Exception as e:
            print(f"An error occurred: {e}")

finally:
    # Save DataFrame to CSV
    df.to_csv('output.csv', index=False)

    # Close the browser
    input("Press Enter to close the browser")
    driver.quit()
   


An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='tableParcelInfoWindowData']"}
  (Session info: chrome=120.0.6099.110); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF726692142+3514994]
	(No symbol) [0x00007FF7262B0CE2]
	(No symbol) [0x00007FF7261576AA]
	(No symbol) [0x00007FF7261A1860]
	(No symbol) [0x00007FF7261A197C]
	(No symbol) [0x00007FF7261E4EE7]
	(No symbol) [0x00007FF7261C602F]
	(No symbol) [0x00007FF7261E28F6]
	(No symbol) [0x00007FF7261C5D93]
	(No symbol) [0x00007FF726194BDC]
	(No symbol) [0x00007FF726195C64]
	GetHandleVerifier [0x00007FF7266BE16B+3695259]
	GetHandleVerifier [0x00007FF726716737+4057191]
	GetHandleVerifier [0x00007FF72670E4E3+4023827]
	GetHandleVerifier [0x00007FF7263E04F9+689705]
	(No symbol) [0x00007FF7262BC048]
	(No symbol) [0x00007FF7262B8044]
	(No symbol) [0

C:\Users\gabbles\AppData\Local\Temp\ipykernel_16452\1012338655.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({



Address: 3304 Daniel Falls Ln
HCAD Account: 1337760010018
Owner Name: BRENNAN WYATT & ELIZABETH
State Class: A1
Appraised Value: $ 368,978.00
Market Value: $ 368,978.00


C:\Users\gabbles\AppData\Local\Temp\ipykernel_16452\1012338655.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({



Address: 3305 Daniel Falls Ln
HCAD Account: 1337760010018
Owner Name: BRENNAN WYATT & ELIZABETH
State Class: A1
Appraised Value: $ 368,978.00
Market Value: $ 368,978.00


...

C:\Users\gabbles\AppData\Local\Temp\ipykernel_16452\1012338655.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({



Address: 3418 Daniel Falls Ln
HCAD Account: 1337760020003
Owner Name: MA KUIZHI
State Class: A1
Appraised Value: $ 378,862.00
Market Value: $ 378,862.00


C:\Users\gabbles\AppData\Local\Temp\ipykernel_16452\1012338655.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\gabbles\AppData\Local\Temp\ipykernel_16452\1012338655.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({



Address: 3419 Daniel Falls Ln
HCAD Account: 1337760010004
Owner Name: RAMIREZ JACQUELINE
State Class: A1
Appraised Value: $ 280,631.00
Market Value: $ 280,631.00


KeyboardInterrupt: Interrupted by user

In [1]:
df_1

NameError: name 'df_1' is not defined

In [9]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.alert import Alert
import time 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd 

# Initialize the webdriver
driver = webdriver.Chrome()
driver.get("https://arcweb.hcad.org/parcelviewer/")
ok_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//td[contains(text(),'OK')]"))
)
ok_button.click()

# Click the HTML for the dropdown thingy
dropdown_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//img[@align='left']"))
)
dropdown_button.click()

# Click on the HTML for the address
address_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//td[@value='address']"))
)
address_button.click()

# Create a list of addresses
addresses = [f"{i} Daniel Falls Ln" for i in range(3419, 3541)]

# Create an empty DataFrame to store the results
columns = ['Address', 'HCAD Account', 'Owner Name', 'State Class', 'Appraised Value', 'Market Value']
df = pd.DataFrame(columns=columns)

try:
    for address_to_search in addresses:
        # Animation 2: click the search button
        search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//input[@id='txtAddress']"))
        )
        search_button.click()

        try:
            # Find the input element within the search bar
            input_element_xpath = "//input[@id='txtAddress']"

            # Use an explicit wait to wait for the input element to be present and clickable
            input_element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, input_element_xpath))
            )

            # Input the address into the search bar
            input_element.clear()
            time.sleep(2)  # Adjust this wait time as needed
            input_element.send_keys(address_to_search)
            input_element.send_keys(Keys.RETURN)  # Simulate hitting the "Enter" key

            # Wait for the search result to load (adjust the timeout as needed)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'divParcelDataScrollContainer')))

            # Check if the table is present
            table_present = EC.presence_of_element_located((By.XPATH, "//*[@id='tableParcelInfoWindowData']"))
            if not table_present(driver):
                print(f"No table found for address: {address_to_search}")
                continue  # Move on to the next address if no table is found

            # Extract information from the specified element
            div_table_xpath = "//*[@id='tableParcelInfoWindowData']"

            for _ in range(3):  # Adjust the number of retries as needed
                try:
                    div_table = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, div_table_xpath))
                    )

                    # Extract specific information from the table
                    hcad_account = div_table.find_element(By.XPATH, ".//td[contains(text(), 'HCAD Account:')]/following-sibling::td").text
                    owner_name = div_table.find_element(By.XPATH, ".//td[contains(text(), 'Owner Name:')]/following-sibling::td").text       
                    state_class = div_table.find_element(By.XPATH, ".//td[contains(text(), 'State Class:')]/following-sibling::td").text
                    appraised_value = div_table.find_element(By.XPATH, ".//td[contains(text(), 'Appraised Value:')]/following-sibling::td").text
                    market_value = div_table.find_element(By.XPATH, ".//td[contains(text(), 'Market Value:')]/following-sibling::td").text

                    # Print the extracted information
                    print("\nAddress:", address_to_search)
                    print("HCAD Account:", hcad_account)
                    print("Owner Name:", owner_name)
                    print("State Class:", state_class)
                    print("Appraised Value:", appraised_value)
                    print("Market Value:", market_value)

                    # Append the data to the DataFrame
                    df = df.append({
                        'Address': address_to_search,
                        'HCAD Account': hcad_account,
                        'Owner Name': owner_name,
                        'State Class': state_class,
                        'Appraised Value': appraised_value,
                        'Market Value': market_value
                    }, ignore_index=True)

                    break  # Exit the retry loop if successful

                except StaleElementReferenceException:
                    print("StaleElementReferenceException occurred. Retrying...")
                    time.sleep(2)  # Adjust this wait time as needed

        except Exception as e:
            print(f"An error occurred: {e}")

finally:
    # Save DataFrame to CSV
    df.to_csv('output.csv', index=False)

    # Close the browser
    input("Press Enter to close the browser")
    driver.quit()
   


An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='tableParcelInfoWindowData']"}
  (Session info: chrome=120.0.6099.129); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF726692142+3514994]
	(No symbol) [0x00007FF7262B0CE2]
	(No symbol) [0x00007FF7261576AA]
	(No symbol) [0x00007FF7261A1860]
	(No symbol) [0x00007FF7261A197C]
	(No symbol) [0x00007FF7261E4EE7]
	(No symbol) [0x00007FF7261C602F]
	(No symbol) [0x00007FF7261E28F6]
	(No symbol) [0x00007FF7261C5D93]
	(No symbol) [0x00007FF726194BDC]
	(No symbol) [0x00007FF726195C64]
	GetHandleVerifier [0x00007FF7266BE16B+3695259]
	GetHandleVerifier [0x00007FF726716737+4057191]
	GetHandleVerifier [0x00007FF72670E4E3+4023827]
	GetHandleVerifier [0x00007FF7263E04F9+689705]
	(No symbol) [0x00007FF7262BC048]
	(No symbol) [0x00007FF7262B8044]
	(No symbol) [0

C:\Users\gabbles\AppData\Local\Temp\ipykernel_16452\872758208.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({



Address: 3421 Daniel Falls Ln
HCAD Account: 1337760020002
Owner Name: LEE VICKY LIN & JOHNNY
State Class: A1
Appraised Value: $ 347,384.00
Market Value: $ 347,384.00


C:\Users\gabbles\AppData\Local\Temp\ipykernel_16452\872758208.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({



Address: 3422 Daniel Falls Ln
HCAD Account: 1337760020002
Owner Name: LEE VICKY LIN & JOHNNY
State Class: A1
Appraised Value: $ 347,384.00
Market Value: $ 347,384.00


...


Address: 3539 Daniel Falls Ln
HCAD Account: 1337760010001
Owner Name: SIMPSON ADRIAN M
State Class: A1
Appraised Value: $ 325,657.00
Market Value: $ 394,103.00


C:\Users\gabbles\AppData\Local\Temp\ipykernel_16452\872758208.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\gabbles\AppData\Local\Temp\ipykernel_16452\872758208.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({



Address: 3540 Daniel Falls Ln
HCAD Account: 1337760010001
Owner Name: SIMPSON ADRIAN M
State Class: A1
Appraised Value: $ 325,657.00
Market Value: $ 394,103.00


KeyboardInterrupt: Interrupted by user

Press Enter to close the browser


df_2

In [14]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.alert import Alert
import time 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd 

# Initialize the webdriver
driver = webdriver.Chrome()
driver.get("https://arcweb.hcad.org/parcelviewer/")
ok_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//td[contains(text(),'OK')]"))
)
ok_button.click()

# Click the HTML for the dropdown thingy
dropdown_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//img[@align='left']"))
)
dropdown_button.click()

# Click on the HTML for the address
address_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//td[@value='address']"))
)
address_button.click()

# Create a list of addresses
addresses = [f"{i} Daniel Falls Ln" for i in range(3540, 3661)]

# Create an empty DataFrame to store the results
columns = ['Address', 'HCAD Account', 'Owner Name', 'State Class', 'Appraised Value', 'Market Value']
df = pd.DataFrame(columns=columns)

try:
    for address_to_search in addresses:
        # Animation 2: click the search button
        search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//input[@id='txtAddress']"))
        )
        search_button.click()

        try:
            # Find the input element within the search bar
            input_element_xpath = "//input[@id='txtAddress']"

            # Use an explicit wait to wait for the input element to be present and clickable
            input_element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, input_element_xpath))
            )

            # Input the address into the search bar
            input_element.clear()
            time.sleep(2)  # Adjust this wait time as needed
            input_element.send_keys(address_to_search)
            input_element.send_keys(Keys.RETURN)  # Simulate hitting the "Enter" key

            # Wait for the search result to load (adjust the timeout as needed)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'divParcelDataScrollContainer')))

            # Check if the table is present
            table_present = EC.presence_of_element_located((By.XPATH, "//*[@id='tableParcelInfoWindowData']"))
            if not table_present(driver):
                print(f"No table found for address: {address_to_search}")
                continue  # Move on to the next address if no table is found

            # Extract information from the specified element
            div_table_xpath = "//*[@id='tableParcelInfoWindowData']"

            for _ in range(3):  # Adjust the number of retries as needed
                try:
                    div_table = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, div_table_xpath))
                    )

                    # Extract specific information from the table
                    hcad_account = div_table.find_element(By.XPATH, ".//td[contains(text(), 'HCAD Account:')]/following-sibling::td").text
                    owner_name = div_table.find_element(By.XPATH, ".//td[contains(text(), 'Owner Name:')]/following-sibling::td").text       
                    state_class = div_table.find_element(By.XPATH, ".//td[contains(text(), 'State Class:')]/following-sibling::td").text
                    appraised_value = div_table.find_element(By.XPATH, ".//td[contains(text(), 'Appraised Value:')]/following-sibling::td").text
                    market_value = div_table.find_element(By.XPATH, ".//td[contains(text(), 'Market Value:')]/following-sibling::td").text

                    # Print the extracted information
                    print("\nAddress:", address_to_search)
                    print("HCAD Account:", hcad_account)
                    print("Owner Name:", owner_name)
                    print("State Class:", state_class)
                    print("Appraised Value:", appraised_value)
                    print("Market Value:", market_value)

                    # Append the data to the DataFrame
                    df = df.append({
                        'Address': address_to_search,
                        'HCAD Account': hcad_account,
                        'Owner Name': owner_name,
                        'State Class': state_class,
                        'Appraised Value': appraised_value,
                        'Market Value': market_value
                    }, ignore_index=True)

                    break  # Exit the retry loop if successful

                except StaleElementReferenceException:
                    print("StaleElementReferenceException occurred. Retrying...")
                    time.sleep(2)  # Adjust this wait time as needed

        except Exception as e:
            print(f"An error occurred: {e}")

finally:
    # Save DataFrame to CSV
    df.to_csv('output.csv', index=False)

    # Close the browser
    input("Press Enter to close the browser")
    driver.quit()
   


An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='tableParcelInfoWindowData']"}
  (Session info: chrome=120.0.6099.129); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF726692142+3514994]
	(No symbol) [0x00007FF7262B0CE2]
	(No symbol) [0x00007FF7261576AA]
	(No symbol) [0x00007FF7261A1860]
	(No symbol) [0x00007FF7261A197C]
	(No symbol) [0x00007FF7261E4EE7]
	(No symbol) [0x00007FF7261C602F]
	(No symbol) [0x00007FF7261E28F6]
	(No symbol) [0x00007FF7261C5D93]
	(No symbol) [0x00007FF726194BDC]
	(No symbol) [0x00007FF726195C64]
	GetHandleVerifier [0x00007FF7266BE16B+3695259]
	GetHandleVerifier [0x00007FF726716737+4057191]
	GetHandleVerifier [0x00007FF72670E4E3+4023827]
	GetHandleVerifier [0x00007FF7263E04F9+689705]
	(No symbol) [0x00007FF7262BC048]
	(No symbol) [0x00007FF7262B8044]
	(No symbol) [0

In [ ]:
df_3

In [18]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.alert import Alert
import time 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd 

# Initialize the webdriver
driver = webdriver.Chrome()
driver.get("https://arcweb.hcad.org/parcelviewer/")
ok_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//td[contains(text(),'OK')]"))
)
ok_button.click()

# Click the HTML for the dropdown thingy
dropdown_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//img[@align='left']"))
)
dropdown_button.click()

# Click on the HTML for the address
address_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//td[@value='address']"))
)
address_button.click()

# Create a list of addresses
addresses = [f"{i} Daniel Falls Ln" for i in range(3662, 3721)]

# Create an empty DataFrame to store the results
columns = ['Address', 'HCAD Account', 'Owner Name', 'State Class', 'Appraised Value', 'Market Value']
df = pd.DataFrame(columns=columns)

try:
    for address_to_search in addresses:
        # Animation 2: click the search button
        search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//input[@id='txtAddress']"))
        )
        search_button.click()

        try:
            # Find the input element within the search bar
            input_element_xpath = "//input[@id='txtAddress']"

            # Use an explicit wait to wait for the input element to be present and clickable
            input_element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, input_element_xpath))
            )

            # Input the address into the search bar
            input_element.clear()
            time.sleep(2)  # Adjust this wait time as needed
            input_element.send_keys(address_to_search)
            input_element.send_keys(Keys.RETURN)  # Simulate hitting the "Enter" key

            # Wait for the search result to load (adjust the timeout as needed)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'divParcelDataScrollContainer')))

            # Check if the table is present
            table_present = EC.presence_of_element_located((By.XPATH, "//*[@id='tableParcelInfoWindowData']"))
            if not table_present(driver):
                print(f"No table found for address: {address_to_search}")
                continue  # Move on to the next address if no table is found

            # Extract information from the specified element
            div_table_xpath = "//*[@id='tableParcelInfoWindowData']"

            for _ in range(3):  # Adjust the number of retries as needed
                try:
                    div_table = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, div_table_xpath))
                    )

                    # Extract specific information from the table
                    hcad_account = div_table.find_element(By.XPATH, ".//td[contains(text(), 'HCAD Account:')]/following-sibling::td").text
                    owner_name = div_table.find_element(By.XPATH, ".//td[contains(text(), 'Owner Name:')]/following-sibling::td").text       
                    state_class = div_table.find_element(By.XPATH, ".//td[contains(text(), 'State Class:')]/following-sibling::td").text
                    appraised_value = div_table.find_element(By.XPATH, ".//td[contains(text(), 'Appraised Value:')]/following-sibling::td").text
                    market_value = div_table.find_element(By.XPATH, ".//td[contains(text(), 'Market Value:')]/following-sibling::td").text

                    # Print the extracted information
                    print("\nAddress:", address_to_search)
                    print("HCAD Account:", hcad_account)
                    print("Owner Name:", owner_name)
                    print("State Class:", state_class)
                    print("Appraised Value:", appraised_value)
                    print("Market Value:", market_value)

                    # Append the data to the DataFrame
                    df = df.append({
                        'Address': address_to_search,
                        'HCAD Account': hcad_account,
                        'Owner Name': owner_name,
                        'State Class': state_class,
                        'Appraised Value': appraised_value,
                        'Market Value': market_value
                    }, ignore_index=True)

                    break  # Exit the retry loop if successful

                except StaleElementReferenceException:
                    print("StaleElementReferenceException occurred. Retrying...")
                    time.sleep(2)  # Adjust this wait time as needed

        except Exception as e:
            print(f"An error occurred: {e}")

finally:
    # Save DataFrame to CSV
    df.to_csv('output.csv', index=False)

    # Close the browser
    input("Press Enter to close the browser")
    driver.quit()
   


An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='tableParcelInfoWindowData']"}
  (Session info: chrome=120.0.6099.129); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF726692142+3514994]
	(No symbol) [0x00007FF7262B0CE2]
	(No symbol) [0x00007FF7261576AA]
	(No symbol) [0x00007FF7261A1860]
	(No symbol) [0x00007FF7261A197C]
	(No symbol) [0x00007FF7261E4EE7]
	(No symbol) [0x00007FF7261C602F]
	(No symbol) [0x00007FF7261E28F6]
	(No symbol) [0x00007FF7261C5D93]
	(No symbol) [0x00007FF726194BDC]
	(No symbol) [0x00007FF726195C64]
	GetHandleVerifier [0x00007FF7266BE16B+3695259]
	GetHandleVerifier [0x00007FF726716737+4057191]
	GetHandleVerifier [0x00007FF72670E4E3+4023827]
	GetHandleVerifier [0x00007FF7263E04F9+689705]
	(No symbol) [0x00007FF7262BC048]
	(No symbol) [0x00007FF7262B8044]
	(No symbol) [0

In [21]:
df_4 = df

In [22]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.alert import Alert
import time 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd 

# Initialize the webdriver
driver = webdriver.Chrome()
driver.get("https://arcweb.hcad.org/parcelviewer/")
ok_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//td[contains(text(),'OK')]"))
)
ok_button.click()

# Click the HTML for the dropdown thingy
dropdown_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//img[@align='left']"))
)
dropdown_button.click()

# Click on the HTML for the address
address_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//td[@value='address']"))
)
address_button.click()

# Create a list of addresses
addresses = [f"{i} Daniel Falls Ln" for i in range(3722, 3830)]

# Create an empty DataFrame to store the results
columns = ['Address', 'HCAD Account', 'Owner Name', 'State Class', 'Appraised Value', 'Market Value']
df = pd.DataFrame(columns=columns)

try:
    for address_to_search in addresses:
        # Animation 2: click the search button
        search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//input[@id='txtAddress']"))
        )
        search_button.click()

        try:
            # Find the input element within the search bar
            input_element_xpath = "//input[@id='txtAddress']"

            # Use an explicit wait to wait for the input element to be present and clickable
            input_element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, input_element_xpath))
            )

            # Input the address into the search bar
            input_element.clear()
            time.sleep(2)  # Adjust this wait time as needed
            input_element.send_keys(address_to_search)
            input_element.send_keys(Keys.RETURN)  # Simulate hitting the "Enter" key

            # Wait for the search result to load (adjust the timeout as needed)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'divParcelDataScrollContainer')))

            # Check if the table is present
            table_present = EC.presence_of_element_located((By.XPATH, "//*[@id='tableParcelInfoWindowData']"))
            if not table_present(driver):
                print(f"No table found for address: {address_to_search}")
                continue  # Move on to the next address if no table is found

            # Extract information from the specified element
            div_table_xpath = "//*[@id='tableParcelInfoWindowData']"

            for _ in range(3):  # Adjust the number of retries as needed
                try:
                    div_table = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, div_table_xpath))
                    )

                    # Extract specific information from the table
                    hcad_account = div_table.find_element(By.XPATH, ".//td[contains(text(), 'HCAD Account:')]/following-sibling::td").text
                    owner_name = div_table.find_element(By.XPATH, ".//td[contains(text(), 'Owner Name:')]/following-sibling::td").text       
                    state_class = div_table.find_element(By.XPATH, ".//td[contains(text(), 'State Class:')]/following-sibling::td").text
                    appraised_value = div_table.find_element(By.XPATH, ".//td[contains(text(), 'Appraised Value:')]/following-sibling::td").text
                    market_value = div_table.find_element(By.XPATH, ".//td[contains(text(), 'Market Value:')]/following-sibling::td").text

                    # Print the extracted information
                    print("\nAddress:", address_to_search)
                    print("HCAD Account:", hcad_account)
                    print("Owner Name:", owner_name)
                    print("State Class:", state_class)
                    print("Appraised Value:", appraised_value)
                    print("Market Value:", market_value)

                    # Append the data to the DataFrame
                    df = df.append({
                        'Address': address_to_search,
                        'HCAD Account': hcad_account,
                        'Owner Name': owner_name,
                        'State Class': state_class,
                        'Appraised Value': appraised_value,
                        'Market Value': market_value
                    }, ignore_index=True)

                    break  # Exit the retry loop if successful

                except StaleElementReferenceException:
                    print("StaleElementReferenceException occurred. Retrying...")
                    time.sleep(2)  # Adjust this wait time as needed

        except Exception as e:
            print(f"An error occurred: {e}")

finally:
    # Save DataFrame to CSV
    df.to_csv('output.csv', index=False)

    # Close the browser
    input("Press Enter to close the browser")
    driver.quit()
   


An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='tableParcelInfoWindowData']"}
  (Session info: chrome=120.0.6099.129); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF726692142+3514994]
	(No symbol) [0x00007FF7262B0CE2]
	(No symbol) [0x00007FF7261576AA]
	(No symbol) [0x00007FF7261A1860]
	(No symbol) [0x00007FF7261A197C]
	(No symbol) [0x00007FF7261E4EE7]
	(No symbol) [0x00007FF7261C602F]
	(No symbol) [0x00007FF7261E28F6]
	(No symbol) [0x00007FF7261C5D93]
	(No symbol) [0x00007FF726194BDC]
	(No symbol) [0x00007FF726195C64]
	GetHandleVerifier [0x00007FF7266BE16B+3695259]
	GetHandleVerifier [0x00007FF726716737+4057191]
	GetHandleVerifier [0x00007FF72670E4E3+4023827]
	GetHandleVerifier [0x00007FF7263E04F9+689705]
	(No symbol) [0x00007FF7262BC048]
	(No symbol) [0x00007FF7262B8044]
	(No symbol) [0

In [24]:
df_5 

In [25]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.alert import Alert
import time 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd 

# Initialize the webdriver
driver = webdriver.Chrome()
driver.get("https://arcweb.hcad.org/parcelviewer/")
ok_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//td[contains(text(),'OK')]"))
)
ok_button.click()

# Click the HTML for the dropdown thingy
dropdown_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//img[@align='left']"))
)
dropdown_button.click()

# Click on the HTML for the address
address_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//td[@value='address']"))
)
address_button.click()

# Create a list of addresses
addresses = [f"{i} Daniel Falls Ln" for i in range(3831, 4000)]

# Create an empty DataFrame to store the results
columns = ['Address', 'HCAD Account', 'Owner Name', 'State Class', 'Appraised Value', 'Market Value']
df = pd.DataFrame(columns=columns)

try:
    for address_to_search in addresses:
        # Animation 2: click the search button
        search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//input[@id='txtAddress']"))
        )
        search_button.click()

        try:
            # Find the input element within the search bar
            input_element_xpath = "//input[@id='txtAddress']"

            # Use an explicit wait to wait for the input element to be present and clickable
            input_element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, input_element_xpath))
            )

            # Input the address into the search bar
            input_element.clear()
            time.sleep(2)  # Adjust this wait time as needed
            input_element.send_keys(address_to_search)
            input_element.send_keys(Keys.RETURN)  # Simulate hitting the "Enter" key

            # Wait for the search result to load (adjust the timeout as needed)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'divParcelDataScrollContainer')))

            # Check if the table is present
            table_present = EC.presence_of_element_located((By.XPATH, "//*[@id='tableParcelInfoWindowData']"))
            if not table_present(driver):
                print(f"No table found for address: {address_to_search}")
                continue  # Move on to the next address if no table is found

            # Extract information from the specified element
            div_table_xpath = "//*[@id='tableParcelInfoWindowData']"

            for _ in range(3):  # Adjust the number of retries as needed
                try:
                    div_table = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, div_table_xpath))
                    )

                    # Extract specific information from the table
                    hcad_account = div_table.find_element(By.XPATH, ".//td[contains(text(), 'HCAD Account:')]/following-sibling::td").text
                    owner_name = div_table.find_element(By.XPATH, ".//td[contains(text(), 'Owner Name:')]/following-sibling::td").text       
                    state_class = div_table.find_element(By.XPATH, ".//td[contains(text(), 'State Class:')]/following-sibling::td").text
                    appraised_value = div_table.find_element(By.XPATH, ".//td[contains(text(), 'Appraised Value:')]/following-sibling::td").text
                    market_value = div_table.find_element(By.XPATH, ".//td[contains(text(), 'Market Value:')]/following-sibling::td").text

                    # Print the extracted information
                    print("\nAddress:", address_to_search)
                    print("HCAD Account:", hcad_account)
                    print("Owner Name:", owner_name)
                    print("State Class:", state_class)
                    print("Appraised Value:", appraised_value)
                    print("Market Value:", market_value)

                    # Append the data to the DataFrame
                    df = df.append({
                        'Address': address_to_search,
                        'HCAD Account': hcad_account,
                        'Owner Name': owner_name,
                        'State Class': state_class,
                        'Appraised Value': appraised_value,
                        'Market Value': market_value
                    }, ignore_index=True)

                    break  # Exit the retry loop if successful

                except StaleElementReferenceException:
                    print("StaleElementReferenceException occurred. Retrying...")
                    time.sleep(2)  # Adjust this wait time as needed

        except Exception as e:
            print(f"An error occurred: {e}")

finally:
    # Save DataFrame to CSV
    df.to_csv('output.csv', index=False)

    # Close the browser
    input("Press Enter to close the browser")
    driver.quit()
   


An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='tableParcelInfoWindowData']"}
  (Session info: chrome=120.0.6099.129); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF726692142+3514994]
	(No symbol) [0x00007FF7262B0CE2]
	(No symbol) [0x00007FF7261576AA]
	(No symbol) [0x00007FF7261A1860]
	(No symbol) [0x00007FF7261A197C]
	(No symbol) [0x00007FF7261E4EE7]
	(No symbol) [0x00007FF7261C602F]
	(No symbol) [0x00007FF7261E28F6]
	(No symbol) [0x00007FF7261C5D93]
	(No symbol) [0x00007FF726194BDC]
	(No symbol) [0x00007FF726195C64]
	GetHandleVerifier [0x00007FF7266BE16B+3695259]
	GetHandleVerifier [0x00007FF726716737+4057191]
	GetHandleVerifier [0x00007FF72670E4E3+4023827]
	GetHandleVerifier [0x00007FF7263E04F9+689705]
	(No symbol) [0x00007FF7262BC048]
	(No symbol) [0x00007FF7262B8044]
	(No symbol) [0

In [35]:
import pandas as pd
dff = pd.concat([df_1, df_2], ignore_index=True)
dff

,Address,HCAD Account,Owner Name,State Class,Appraised Value,Market Value
0,3303 Daniel Falls Ln,1337760010018,BRENNAN WYATT & ELIZABETH,A1,"$ 368,978.00","$ 368,978.00"
1,3304 Daniel Falls Ln,1337760010018,BRENNAN WYATT & ELIZABETH,A1,"$ 368,978.00","$ 368,978.00"
2,3305 Daniel Falls Ln,1337760010018,BRENNAN WYATT & ELIZABETH,A1,"$ 368,978.00","$ 368,978.00"
3,3306 Daniel Falls Ln,1337760010018,BRENNAN WYATT & ELIZABETH,A1,"$ 368,978.00","$ 368,978.00"
4,3307 Daniel Falls Ln,1337760010017,PHAM KIM A,A1,"$ 331,262.00","$ 331,262.00"
...,...,...,...,...,...,...
233,3536 Daniel Falls Ln,1337760010001,SIMPSON ADRIAN M,A1,"$ 325,657.00","$ 394,103.00"
234,3537 Daniel Falls Ln,1337760010001,SIMPSON ADRIAN M,A1,"$ 325,657.00","$ 394,103.00"
235,3538 Daniel Falls Ln,1337760010001,SIMPSON ADRIAN M,A1,"$ 325,657.00","$ 394,103.00"
236,3539 Daniel Falls Ln,1337760010001,SIMPSON ADRIAN M,A1,"$ 325,657.00","$ 394,103.00"


In [38]:
df_without_duplicates = dff.drop_duplicates(subset=df.columns[1:])

df_without_duplicates

,Address,HCAD Account,Owner Name,State Class,Appraised Value,Market Value
0,3303 Daniel Falls Ln,1337760010018,BRENNAN WYATT & ELIZABETH,A1,"$ 368,978.00","$ 368,978.00"
4,3307 Daniel Falls Ln,1337760010017,PHAM KIM A,A1,"$ 331,262.00","$ 331,262.00"
5,3308 Daniel Falls Ln,1337760020011,GUERRA FRANCISCO V ANAYA,A1,"$ 246,930.00","$ 298,758.00"
8,3311 Daniel Falls Ln,1337760010016,KUDINOV ROMAN,A1,"$ 362,000.00","$ 362,000.00"
9,3312 Daniel Falls Ln,1337760020010,CHEN JIN XIU & SONGTAO,A1,"$ 312,773.00","$ 312,773.00"
12,3315 Daniel Falls Ln,1337760010015,PAREDES JOSE & EDELMIRA,A1,"$ 303,751.00","$ 366,906.00"
15,3318 Daniel Falls Ln,1337760010014,JIANG YILAN,A1,"$ 383,926.00","$ 383,926.00"
17,3320 Daniel Falls Ln,1337760020008,SKELTON STACY ELIZABETH,A1,"$ 276,202.00","$ 276,202.00"
20,3323 Daniel Falls Ln,1337760010013,LAZARO MARIA C,A1,"$ 309,040.00","$ 373,035.00"
24,3327 Daniel Falls Ln,1337760010012,AMIRI WAHIDULLAH,A1,"$ 347,090.00","$ 347,090.00"
